<div class=title>
<strong><center><font size="3">DAY 7 - IYKRA</font></center>
<hr>
<center><font size="5"><strong>Practice Case 02</strong></font></center>
<center><font size="4"><strong>Exploratory Data Analysis (EDA) Use Case</strong></font></center>
<hr>
<p style="text-align:center">Author</p>
<center>Joshua Effendi</center>
<p style="text-align:center">Date:</p>
<center>29 October 2019</center></strong>
</div>

<font size="3"><strong>Practice Case 2</strong></font>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns 
from tqdm import tqdm_notebook as tqdm
%matplotlib inline
import scipy.stats as sct
from sklearn import preprocessing
from sklearn import cluster
import plotly.offline as py
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

from IPython.core.display import HTML
def css_styling():
    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)
css_styling()

<font size="3"><strong>Questions</strong></font>
1. Describe each columns (data types, unique values/range of values, missing values, etc.)
2. Is the data fit for analysis? If not, what wrangling steps should be done?
3. From the data, what can you describe about their sample of customers?
4. Are they potentially “lucrative” customers? If so, how many of them are and why?
5. What are the difference between customers who are interested in the product and those who don’t?

In [ ]:
df = pd.read_csv('bank-marketing-full.csv',sep=';')
df

<font size="3"><strong>Data Describe Types</strong></font>


In [ ]:
df.info()

<div class=cbox>
<strong>As seen above, there are three types of data divided into two groups, which is numerical data in which consist of float and integer data and categorical data in which consist of object. There are 10 numerical features divided into <font color="red">5 Float</font> and <font color="red">5 Integer</font>. Lastly there are 11 categorical data in which data type is <font color="red">Object</font></strong>
</div

In [ ]:
df = pd.concat([df,pd.get_dummies(df['y'], prefix = 'bool').drop(columns='bool_no')], axis = 1)
df.head()

<div class=cbox>
    <b>Add numerical data for y feature to <font color='red'>boolean</font> to accomodate the correlation figure.</b>
</div>

In [ ]:
numerical_data = df.select_dtypes(include = [np.number]).columns
categorical_data = df.select_dtypes(exclude = [np.number]).columns

<font size="2"><strong>Divide the columns by it's type, categorical and numerical.</strong></font>

<font size="3"><strong>Data Range/Unique Values</strong></font>

In [ ]:
df[numerical_data].describe()

<div class=cbox>
<b>As seen above is the data descriptions for the numerical data and also the <font color='red'>range</font> of the data</b>
</div>

In [ ]:
df[categorical_data].describe()

<div class=cbox>
<b>As seen above is the data descriptions for the categorical data and also the <font color='red'>unique</font> values of the data.</b>
</div>

<font size="3"><strong>Missing Values</stong></font>

In [ ]:
df.isnull().sum().sort_values(ascending = False)

<div class=cbox>
    <b>No missing values in the data as no need to manipulate the data for the missing values. Except the 999 in data as it is stated as missing values.</b>
</div>

<b>Normalized Relative Frequency of Target</b>

In [ ]:
for col in categorical_data:
    plt.figure(figsize=(10,4))
    
    pos_counts = df.loc[df.y.values == 'yes', col].value_counts() 
    neg_counts = df.loc[df.y.values == 'no', col].value_counts()
    
    all_counts = list(set(list(pos_counts.index) + list(neg_counts.index)))
    
    freq_pos = (df.y.values == 'yes').sum()
    freq_neg = (df.y.values == 'no').sum()
    
    pos_counts = pos_counts.to_dict()
    neg_counts = neg_counts.to_dict()
    
    all_index = list(all_counts)
    all_counts = [pos_counts.get(k, 0) / freq_pos - neg_counts.get(k, 0) / freq_neg for k in all_counts]

    sns.barplot(all_counts, all_index)
    plt.title(col)
    plt.tight_layout()

<div class=cbox>
    <b>There are a lot of unknown data in which have to be handled.</b>
</div>

<font size="3"><strong>Imputation Data - Inferring</strong></font>

In [ ]:
s_cat_dat = ['education','job','housing','loan']
for var in s_cat_dat:
    df[var + '_un'] = (df[var] == 'unknown').astype(int)

In [ ]:
def cross_tab(df,f1,f2):
    jobs=list(df[f1].unique())
    edu=list(df[f2].unique())
    dataframes=[]
    for e in edu:
        dfe=df[df[f2]==e]
        dfejob=dfe.groupby(f1).count()[f2]
        dataframes.append(dfejob)
    xx=pd.concat(dataframes,axis=1)
    xx.columns=edu
    xx=xx.fillna(0)
    return xx

In [ ]:
cross_tab(df,'job','education')

In [ ]:
df['job'][df['age']>60].value_counts().reset_index().set_index('index')

<div class=cbox>
    <b>Inferring: Education, From: Job. From the cross-tabulation data we can infer that people with management job usually have an university education background, similarly with service and high school and housemaid with basic.4y.
<hr>
    Inferring: Job, From: Education. Education of basic.4y, basic.6y, and basic.9y usually have a job blue-collar, similarly with professional.course have a job of technician.
<hr>
    Inferring: Job, From: Age. People with age more than 60 years old have a retired job. </b>
</div>

<b>Replace Data with the Cross-tab Data</b>

In [ ]:
df.loc[(df['age']>60) & (df['job']=='unknown'), 'job'] = 'retired'
df.loc[(df['education']=='unknown') & (df['job']=='management'), 'education'] = 'university.degree'
df.loc[(df['education']=='unknown') & (df['job']=='services'), 'education'] = 'high.school'
df.loc[(df['education']=='unknown') & (df['job']=='housemaid'), 'education'] = 'basic.4y'
df.loc[(df['job'] == 'unknown') & (df['education']=='basic.4y'), 'job'] = 'blue-collar'
df.loc[(df['job'] == 'unknown') & (df['education']=='basic.6y'), 'job'] = 'blue-collar'
df.loc[(df['job'] == 'unknown') & (df['education']=='basic.9y'), 'job'] = 'blue-collar'
df.loc[(df['job']=='unknown') & (df['education']=='professional.course'), 'job'] = 'technician'

<b>Check cross-tab Job and Housing, Loan</b>

In [ ]:
cross_tab(df,'job','housing')

<div class=cbox>
    <b>It has similar split between yes and no in house by jobs.</b>
</div>

In [ ]:
cross_tab(df,'job','loan')

<div class=cbox>
    <b>We can further clean the data by looking at the cross-tab of job and loan.</b>
</div>

In [ ]:
def fillhousing(df,jobhousing):
    jobs=['housemaid','services','admin.','blue-collar','technician','retired','management','unemployed','self-employed','entrepreneur','student']
    house=["no","yes"]
    for j in jobs:
        ind=df[np.logical_and(np.array(df['housing']=='unknown'),np.array(df['job']==j))].index
        mask=np.random.rand(len(ind))<((jobhousing.loc[j]['no'])/(jobhousing.loc[j]['no']+jobhousing.loc[j]['yes']))
        ind1=ind[mask]
        ind2=ind[~mask]
        df.loc[ind1,"housing"]='no'
        df.loc[ind2,"housing"]='yes'
    return df

In [ ]:
def fillloan(df,jobloan):
    jobs=['housemaid','services','admin.','blue-collar','technician','retired','management','unemployed','self-employed','entrepreneur','student']
    loan=["no","yes"]
    for j in jobs:
        ind=df[np.logical_and(np.array(df['loan']=='unknown'),np.array(df['job']==j))].index
        mask=np.random.rand(len(ind))<((jobloan.loc[j]['no'])/(jobloan.loc[j]['no']+jobloan.loc[j]['yes']))
        ind1=ind[mask]
        ind2=ind[~mask]
        df.loc[ind1,"loan"]='no'
        df.loc[ind2,"loan"]='yes'
    return df

In [ ]:
jobhousing=cross_tab(df,'job','housing')
jobloan=cross_tab(df,'job','loan')
df=fillhousing(df,jobhousing)
df=fillloan(df,jobloan)

<b>Numerical Data Imputation</b>

<b>Missing values for pdays for 999</b>

In [ ]:
pd.crosstab(df['pdays'],df['poutcome'], values=df['age'], aggfunc='count', normalize=True)

<div class=cbox>
    <b>It is better to have a categorical data for pdays.</b>
</div>

In [ ]:
df['pdays_missing'] = 0
df['pdays_less_5'] = 0
df['pdays_greater_15'] = 0
df['pdays_bet_5_15'] = 0
df['pdays_missing'][df['pdays'] == 999] = 1
df['pdays_less_5'][df['pdays']<5] = 1
df['pdays_greater_15'][(df['pdays'] > 15) & (df['pdays'] < 999)] = 1
df['pdays_bet_5_15'][(df['pdays'] >= 5) & (df['pdays'] <= 15)]= 1
df_pdays_d = df.drop('pdays', axis=1)
df_pdays_d.head()

In [ ]:
df_dum = pd.get_dummies(df_pdays_d)
df_dum.head()

In [ ]:
def dropfeature(df,f):
    df=df.drop(f,axis=1)
    return df

In [ ]:
features_dropped = ['default_no','housing_no','loan_no','y_no','marital_single','contact_cellular',
                    'education_unknown','job_unknown','housing_unknown','loan_unknown', 'pdays_less_5']
df_clean = dropfeature(df_dum, features_dropped)
df_clean.head()

In [ ]:
numerical_data = df.select_dtypes(include = [np.number]).columns
categorical_data = df.select_dtypes(exclude = [np.number]).columns

<font size="3"><strong>Data Fitting for Analysis</strong></font>

<b>There are features in which have outliers data, that need to be adjusted.</b>

In [ ]:
for i in numerical_data:
    print("IQR for feature {} is {} with minimal value allowed is {} and maximal value allowed is {}.\
    ".format(i.upper(), np.round(sct.iqr(df[i]),2), np.round(df[i].quantile(0.25) - 1.5 * sct.iqr(df[i]),2), \
             np.round(df[i].quantile(0.75) + 1.5 * sct.iqr(df[i]),2)))
    print('Minimal value of data: {} and Maximal value of data: {}\n'.format(np.round(df[i].min(),2), \
                                                                             np.round(df[i].max(),2)))

<div class=cbox>
    <b>As seen from above, there are lots of data in which are outlier so we need to do some cleansing</b>
<div>

<font size="3"><strong>Data Numerical Correlation for Analysis</strong></font>

In [ ]:
corr = df.corr(method = 'pearson')
mask = np.array(corr)
mask[np.tril_indices_from(mask)] = False
fig, ax = plt.subplots(figsize = (20,18))
fig.set_size_inches(20,20)
sns.heatmap(corr, mask = mask, vmax = 0.9, square = True, annot = True)

<div class=cbox>
    <b>As seen the heatmap graph, there are multiple correality, it is wise to avoid features with high correality for modeling</b>
</b>

In [ ]:
np.abs(df.corr(method= 'pearson')['bool_yes']).sort_values(ascending=False)[1:11]

<div class=cbox>
    <b>As seen above, the highest result of correlation for target bool_yes is duration. We could create models with these features, but as there are multiple correality in the features, it is wise to drop on some features to avoid high correality</b>
</div>

<font size="3"><strong>Explanatory Data Analysis</stong></font>

<b>Categorical Data Plot</b>

In [ ]:
for col in categorical_data:
    plt.figure(figsize=(10,4))
    sns.barplot(df[col].value_counts().values, df[col].value_counts().index)
    plt.title(col)
    plt.tight_layout()

<b>Numerical Data Plot</b>

In [ ]:
for col in numerical_data:
    if len(df[col].unique()) <= 5:
        bin1 = 2
    elif len(df[col].unique()) <= 10:
        bin1 = 5
    elif len(df[col].unique()) <= 50:
        bin1 = 10
    else:
        bin1 = 100
    fig = px.histogram(df, x = col, nbins=bin1)
    fig.show()

In [ ]:
numerical_data_c = df_clean.select_dtypes(include = [np.number]).columns
categorical_data_c = df_clean.select_dtypes(exclude = [np.number]).columns

In [ ]:
for col in numerical_data_c:
    if len(df_clean[col].unique()) <= 5:
        bin1 = 2
    elif len(df_clean[col].unique()) <= 10:
        bin1 = 5
    elif len(df_clean[col].unique()) <= 50:
        bin1 = 10
    else:
        bin1 = 100
    fig = px.histogram(df_clean, x = col, nbins=bin1)
    fig.show()

<b>Cluster Data</br>

In [ ]:
combined_data = df[['age','nr.employed']].as_matrix()
combined_data_scaled = preprocessing.scale(combined_data)

kmeans = cluster.KMeans(n_clusters = 3)
kmeans.fit(combined_data_scaled)
y_pred = kmeans.predict(combined_data_scaled)

plt.scatter(combined_data_scaled[:, 0], combined_data_scaled[:, 1], c = y_pred)
plt.xlabel('Scaled Age')
plt.ylabel('Scaled  number of employees')
plt.show()

<div class=cbox>
    <b>The graph shows the cluster of age range and particular job employee.</b>
</div>

<b>Plot Divide Boolean Yes</b>

In [ ]:
for col in categorical_data:
    plt.figure(figsize=(8,4))
    sns.countplot(x=col,data=df,hue='y')
    plt.title(col)
    plt.tight_layout()

<font size='3'><b>Data Analysis</b></font>

<div class=cbox>
    <b>The sample data is not good enough as there are lots of unknown data and need to be engineer to be a good sample data.
<hr>
    There are always "lucrative" customers as there are lot of potential customer even though the majority of the customers are saying no. But as we do not know the current income of the customers we could not really say that if that customers are really lucrative or not. As lucrative also based on the customers risk also and the data there is not any info for customers' risk, so we could not also infer if they are lucrative or not. But if could create a model, we could tell the lucrative customers but as this is only EDA, so there is no model.
<hr>    
    From the data we can infer that customers whom are married that own a house and their poutcome are non-existence has a higher chance of saying yes. Also customers that have a success previously had a higher chance to say yes also. Based on percentage, customers whom had technician as their job had a higer chance also to say yes.</b>
</div>